In [1]:
# load dataset used in 3.2
import pandas as pd

df = pd.read_csv("/Users/frankdong/Documents/Analytics Local/williams-racing-strategies/processed_data/driver-lap-times-validated.csv")

print(df.head())

   Unnamed: 0  race_id  gp_year             gp_name  gp_round  driver_id  \
0           0      930     2015  Spanish Grand Prix         5         13   
1           1      930     2015  Spanish Grand Prix         5         13   
2           2      930     2015  Spanish Grand Prix         5         13   
3           3      930     2015  Spanish Grand Prix         5         13   
4           4      930     2015  Spanish Grand Prix         5         13   

    driver_name rookie_or_experienced  lap_number  lap_time  lap_time_ms  
0  Felipe Massa           experienced           1  1:41.963       101963  
1  Felipe Massa           experienced           2  1:35.272        95272  
2  Felipe Massa           experienced           3  1:33.305        93305  
3  Felipe Massa           experienced           4  1:33.263        93263  
4  Felipe Massa           experienced           5  1:33.557        93557  
